In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')
from component.model import BenefitModel
from component.model import ConstraintModel
from component.model import CostModel
from component.model import SeplanAoi

from component.scripts.seplan import Seplan


from sepal_ui.mapping import SepalMap, InspectorControl
from component.scripts.seplan import _percentile
from sepal_ui.scripts.utils import init_ee
init_ee()

<IPython.core.display.Javascript object>

ResizeTrigger()

In [3]:
seplan_aoi = SeplanAoi()
benefit_model = BenefitModel()
constraint_model = ConstraintModel()
cost_model = CostModel()
seplan_model = Seplan(
    seplan_aoi,
    benefit_model,
    constraint_model,
    cost_model,
)

In [4]:
# Check Afghanistan
seplan_aoi.aoi_model._from_admin(admin="1")
aoi = seplan_aoi.aoi_model.feature_collection

In [5]:
benefit_list = seplan_model.get_benefits_list()
benefit_list

[[<ee.image.Image at 0x7d36004b0fa0>, 'biodiversity_intactness'],
 [<ee.image.Image at 0x7d36004b36d0>, 'endangered_species'],
 [<ee.image.Image at 0x7d3559da35e0>, 'ground_carbon'],
 [<ee.image.Image at 0x7d3559da3610>, 'forest_job'],
 [<ee.image.Image at 0x7d3559da3190>, 'woodfuel_harvest'],
 [<ee.image.Image at 0x7d3559da23b0>, 'plantation_growth_rates']]

In [6]:
ee_growth_rates = benefit_list[-1][0].add(1)

In [7]:
from component.scripts.seplan import quintiles
import ee

In [8]:
ee_growth_rates.reduceRegion(
    reducer=ee.Reducer.frequencyHistogram(),
    geometry=aoi.geometry(),
    scale=100,
    maxPixels=1e13,
).getInfo()

EEException: Image.load: Image asset 'projects/se-plan/assets/inputLayers/yields-4326-edited' not found (does not exist or caller does not have access).

In [ ]:
ee_growth_rates.reduceRegion(
    reducer=ee.Reducer.percentile(percentiles=[20, 40, 60, 80]),
    geometry=aoi.geometry(),
    tileScale=2,
    scale=100,
    maxPixels=1e13,
).getInfo()

In [ ]:
quintiles(ee_growth_rates, aoi).reduceRegion(
    reducer=ee.Reducer.frequencyHistogram(),
    geometry=aoi.geometry(),
    scale=100,
    maxPixels=1e13,
).getInfo()

In [ ]:
# Caluclate quintiles for all default assets on benefits

benefits = [ee.Image(benefit_id) for benefit_id in seplan_model.benefit_model.assets]

## Define variables


In [ ]:
ee_image = ee.Image(benefits[0])
ee_aoi = seplan_aoi.feature_collection
scale = ee_image.projection().nominalScale().multiply(2)

## Test with reduceRegion


In [ ]:
band_name = ee.String(ee_image.bandNames().get(0))
quintiles_dict = ee_image.reduceRegion(
    reducer=ee.Reducer.percentile(percentiles=[20, 40, 60, 80]),
    geometry=ee_aoi,
    tileScale=2,
    scale=scale,
    maxPixels=1e13,
)

In [ ]:
quintiles_names = ee.List(["p20", "p40", "p60", "p80"]).map(
    lambda quintile_name: band_name.cat("_").cat(ee.String(quintile_name))
)

In [ ]:
quintiles_names

In [ ]:
import json

json.loads(quintiles_names)

### Reclassify quintiles


In [ ]:
low = ee.Number(quintiles_dict.get(quintiles_names.get(0)))
lowmed = ee.Number(quintiles_dict.get(quintiles_names.get(1)))
highmed = ee.Number(quintiles_dict.get(quintiles_names.get(2)))
high = ee.Number(quintiles_dict.get(quintiles_names.get(3)))

In [ ]:
quintiles = (
    ee.Image(0)
    .where(ee_image.lte(low), 1)
    .where(ee_image.gt(low).And(ee_image.lte(lowmed)), 2)
    .where(ee_image.gt(lowmed).And(ee_image.lte(highmed)), 3)
    .where(ee_image.gt(highmed).And(ee_image.lte(high)), 4)
    .where(ee_image.gt(high), 5)
)

In [ ]:
quintiles.getInfo()

## Test with reduceRegionS


In [ ]:
from sepal_ui.mapping import SepalMap

map_ = SepalMap()

In [ ]:
map_.add_ee_layer(ee_aoi)
map_.centerObject(ee_aoi)
map_

In [ ]:
# get the number of features of the ee_aoi

ee_aoi.size().getInfo()

In [ ]:
quintile_collection = ee_image.reduceRegions(
    collection=ee_aoi,
    reducer=ee.Reducer.percentile(
        percentiles=[20, 40, 60, 80],
        outputNames=["low", "lowmed", "highmed", "high"],
    ),
    tileScale=2,
    scale=scale,
)
labels = ["high", "low", "lowmed", "highmed"]
# only use features that have non null quintiles
valid_quintiles = quintile_collection.filter(ee.Filter.notNull(labels))

In [ ]:
quintile_values = {}
for f_id, feature in enumerate(valid_quintiles.getInfo()["features"]):
    quintile_values[f_id] = {}.setdefault(f_id, {})
    for prop, val in feature["properties"].items():
        if prop in labels:
            quintile_values[f_id][prop] = val

In [ ]:
quintile_values